In [ ]:
import sys 
import bagpy
from bagpy import bagreader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import os
from scipy import stats


bag_stationary_open = bagreader('/home/sha/Docs/northeastern/RSN/EECE5554/gnss/data/stationary_readings_open.bag)
gps_stationary_open = bag_stationary_open.message_by_topic('/gps_data')
data_stationary_open = pd.read_csv(gps_stationary_open)

bag_stationary_occluded = bagreader('/home/sha/Docs/northeastern/RSN/EECE5554/gnss/data/stationary_readings_occluded.bag')
gps_stationary_occluded = bag_stationary_occluded.message_by_topic('/gps_data')
data_stationary_occluded = pd.read_csv(gps_stationary_occluded)

bag_dynamic_open = bagreader('/home/sha/Docs/northeastern/RSN/EECE5554/gnss/data/dynamic_readings_open.bag')
gps_dynamic_open = bag_dynamic_open.message_by_topic('/gps_data')
data_dynamic_open = pd.read_csv(gps_dynamic_open)

bag_dynamic_occluded = bagreader('/home/sha/Docs/northeastern/RSN/EECE5554/gnss/data/dynamic_readings_occluded.bag')
gps_dynamic_occluded = bag_dynamic_open.message_by_topic('/gps_data')
data_dynamic_occluded = pd.read_csv(gps_dynamic_occluded)

def process_data(data):
    data['Easting_Offset'] = data['utm_easting'] - data['utm_easting'].iloc[0]
    data['Northing_Offset'] = data['utm_northing'] - data['utm_northing'].iloc[0]
    centroid_easting = data['Easting_Offset'].mean()
    centroid_northing = data['Northing_Offset'].mean()
    data['Deviation_Easting'] = data['Easting_Offset'] - centroid_easting
    data['Deviation_Northing'] = data['Northing_Offset'] - centroid_northing
    data['Distance_Centroid'] = np.sqrt(data['Deviation_Easting']**2 + data['Deviation_Northing']**2)
    return data, centroid_easting, centroid_northing

data_stationary_open, centroid_easting_open, centroid_northing_open = process_data(data_stationary_open)
data_stationary_occluded, centroid_easting_occluded, centroid_northing_occluded = process_data(data_stationary_occluded)
data_dynamic_open, centroid_easting_dynamic, centroid_northing_dynamic = process_data(data_dynamic_open)

data_stationary_open['Time'] = data_stationary_open['Time'] - data_stationary_open['Time'].iloc[0]
data_stationary_occluded['Time'] = data_stationary_occluded['Time'] - data_stationary_occluded['Time'].iloc[0]
time_open = data_stationary_open['Time'].to_numpy()
altitude_open = data_stationary_open['altitude'].to_numpy()
time_occluded = data_stationary_occluded['Time'].to_numpy()
altitude_occluded = data_stationary_occluded['altitude'].to_numpy()

def calculate_euclidean_distance(data, centroid_easting, centroid_northing):
    data['Euclidean_distance'] = np.sqrt(
        (data['Easting_Offset'] - centroid_easting) ** 2 +
        (data['Northing_Offset'] - centroid_northing) ** 2
    )
    return data

def plot_histogram_open(data):
    plt.figure(figsize=(8, 6))
    plt.hist(data['Euclidean_distance'], bins=30)
    plt.title('Histogram of Euclidean Distances to Centroid (open)')
    plt.xlabel('Distance (m)')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()
def plot_histogram_occluded(data):
    plt.figure(figsize=(8, 6))
    plt.hist(data['Euclidean_distance'], bins=30)
    plt.title('Histogram of Euclidean Distances to Centroid (Occluded)')
    plt.xlabel('Distance (m)')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()


#plot 1
def plot_northing_easting(data_open, data_occluded, centroid_open, centroid_occluded):
    plt.figure(figsize=(10, 8))
    plt.scatter(data_open['Easting_Offset'], data_open['Northing_Offset'], label='Open', marker='o', alpha=0.7)
    plt.scatter(data_occluded['Easting_Offset'], data_occluded['Northing_Offset'], label='Occluded', marker='x', alpha=0.7)
    plt.scatter(centroid_open[0], centroid_open[1], label='Centroid Open', marker='D', color='blue', s=100)
    plt.scatter(centroid_occluded[0], centroid_occluded[1], label='Centroid Occluded', marker='D', color='orange', s=100)
    plt.annotate(f'Centroid Open\n({centroid_open[0]:.2f}, {centroid_open[1]:.2f})',
                 xy=(centroid_open[0], centroid_open[1]),
                 xytext=(10, 10),
                 textcoords='offset points',
                 fontsize=10,
                 arrowprops=dict(arrowstyle='->', color='blue'),
                 color='blue')
    plt.annotate(f'Centroid Occluded\n({centroid_occluded[0]:.2f}, {centroid_occluded[1]:.2f})',
                 xy=(centroid_occluded[0], centroid_occluded[1]),
                 xytext=(-100, -30),
                 textcoords='offset points',
                 fontsize=10,
                 arrowprops=dict(arrowstyle='->', color='orange'),
                 color='orange')
    plt.title('Stationary Northing vs. Easting Scatterplot with Centroid Values Marked')
    plt.xlabel('Offset Easting (m)')
    plt.ylabel('Offset Northing (m)')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_northing_easting(
    data_stationary_open, data_stationary_occluded,
    (centroid_easting_open, centroid_northing_open),
    (centroid_easting_occluded, centroid_northing_occluded)
)

#plot 2
plt.figure(figsize=(10, 6))
plt.plot(
    time_open,
    altitude_open,
    label='Open',
    alpha=0.7,
    marker='o'
)
plt.plot(
    time_occluded,
    altitude_occluded,
    label='Occluded',
    alpha=0.7,
    marker='x'
)
plt.title('Stationary Altitude vs. Time')
plt.xlabel('Time (seconds)')
plt.ylabel('Altitude (meters)')
plt.legend()
plt.grid(True)
plt.show()
#plot 3a & 3b
data_stationary_open = calculate_euclidean_distance(data_stationary_open, centroid_easting_open, centroid_northing_open)
data_stationary_occluded = calculate_euclidean_distance(data_stationary_occluded, centroid_easting_occluded, centroid_northing_occluded)
plot_histogram_open(data_stationary_open)
plot_histogram_occluded(data_stationary_occluded)

#plot 4

bag_dynamic_open = bagreader('/home/sha/RSN/test/data/data1/dynamic_readings_open.bag')
gps_dynamic_open = bag_dynamic_open.message_by_topic('/gps_data')
data_dynamic_open = pd.read_csv(gps_dynamic_open)

bag_dynamic_occluded = bagreader('/home/sha/RSN/test/data/data2/dynamic_readings_occluded.bag')
gps_dynamic_occluded = bag_dynamic_open.message_by_topic('/gps_data')
data_dynamic_occluded = pd.read_csv('/home/sha/RSN/test/data/data2/dynamic_readings_occluded/1gps_data.csv')

# Function to subtract the first point
def subtract_first_point(data):
    first_easting = data['utm_easting'].iloc[0]
    first_northing = data['utm_northing'].iloc[0]
    data['Easting_Offset'] = data['utm_easting'] - first_easting
    data['Northing_Offset'] = data['utm_northing'] - first_northing
    return data

def calculate_line_of_best_fit(data):
    # Ensure the necessary columns exist
    if 'Easting_Offset' not in data.columns or 'Northing_Offset' not in data.columns:
        raise ValueError("Data must contain 'Easting_Offset' and 'Northing_Offset' columns.")

    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        data['Easting_Offset'], data['Northing_Offset']
    )
    
    # Calculate best fit values
    data['Best_fit_northing'] = slope * data['Easting_Offset'] + intercept
    return data, slope, intercept

def plot_northing_easting_moving(data_open, data_occluded, slope_open, intercept_open, slope_occluded, intercept_occluded):
    plt.figure(figsize=(12, 8))

    # Plot open data
    plt.scatter(
        data_open['Easting_Offset'],
        data_open['Northing_Offset'],
        label='Open Data',
        marker='o',
        color='blue',
        alpha=0.7,
    )
    if 'Best_fit_northing' in data_open.columns:
        plt.plot(
            data_open['Easting_Offset'].values,  # Ensure this is a 1D array
            data_open['Best_fit_northing'].values,  # Ensure this is a 1D array
            label='Best Fit Open',
            linestyle='--',
            color='blue',
        )

    # Plot occluded data
    plt.scatter(
        data_occluded['Easting_Offset'],
        data_occluded['Northing_Offset'],
        label='Occluded Data',
        marker='x',
        color='orange',
        alpha=0.7,
    )
    if 'Best_fit_northing' in data_occluded.columns:
        plt.plot(
            data_occluded['Easting_Offset'].values,  # Ensure this is a 1D array
            data_occluded['Best_fit_northing'].values,  # Ensure this is a 1D array
            label='Best Fit Occluded',
            linestyle='--',
            color='orange',
        )

    # Add slope/intercept text
    plt.text(0.05, 0.95, f'Open Best Fit: y = {slope_open:.4f}x + {intercept_open:.4f}', transform=plt.gca().transAxes, fontsize=10, color='blue', verticalalignment='top')
    plt.text(0.05, 0.90, f'Occluded Best Fit: y = {slope_occluded:.4f}x + {intercept_occluded:.4f}', transform=plt.gca().transAxes, fontsize=10, color='orange', verticalalignment='top')

    # Plot formatting
    plt.title('Moving Northing vs. Easting Scatterplot with Line of Best Fit')
    plt.xlabel('Offset Easting (m)')
    plt.ylabel('Offset Northing (m)')
    plt.legend()
    plt.grid(True)
    plt.show()

data_dynamic_open = subtract_first_point(data_dynamic_open)
data_dynamic_occluded = subtract_first_point(data_dynamic_occluded)
data_dynamic_open, slope_open, intercept_open = calculate_line_of_best_fit(data_dynamic_open)
data_dynamic_occluded, slope_occluded, intercept_occluded = calculate_line_of_best_fit(data_dynamic_occluded)

plot_northing_easting_moving(
    data_dynamic_open,
    data_dynamic_occluded,
    slope_open,
    intercept_open,
    slope_occluded,
    intercept_occluded,
)

# Create time index
data_dynamic_open['Time'] = np.arange(len(data_dynamic_open))
data_dynamic_occluded['Time'] = np.arange(len(data_dynamic_occluded))

# Plot altitude vs. time
plt.figure(figsize=(10, 6))
plt.plot(
    data_dynamic_open['Time'].values,  # Ensure this is a 1D array
    data_dynamic_open['altitude'].values,  # Ensure this is a 1D array
    label='Open',
    marker='o',
    linestyle='-',
    markersize=4,
)

plt.plot(
    data_dynamic_occluded['Time'].values,  # Ensure this is a 1D array
    data_dynamic_occluded['altitude'].values,  # Ensure this is a 1D array
    label='Occluded',
    marker='x',
    linestyle='-',
    markersize=4,
)

plt.title('Moving Altitude vs. Time')
plt.xlabel('Time')
plt.ylabel('Altitude (m)')
plt.legend()
plt.grid(True)
plt.show()